<a href="https://colab.research.google.com/github/PUC-RecSys-Class/RecSysPUC-2020/blob/master/practicos/pyRecLab_SlopeOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  Clemente Sepúlveda

## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:** \\
Este recomendador es un algoritmo de Filtrado Colaborativo debido a que necesita ratings que han hecho otros usuarios para poder predecir (o recomendar) algún item a un usuario en específico (por eso es 'colaborativo'). \\
 Los datos que se necesitan son ratings $r_{ui}$, que significa un rating que el usuario $u$ hizo al item $i$. Esto se puede visualizar como una matriz con los usuarios en las filas y los items en las columnas. Lo que hace slope one es encontrar las diferencias entre dos ratings de dos items, para muchos usuarios. Esto se puede ver como encontrar el $b$ de la función $f(x) = x + b$, $x$ siendo el rating de un item1, $f(x)$ siendo la predicción para un item2, y b siendo el promedio de las diferencia entre item2 e item1 para los usuarios que dieron ratings a esos items.\\
Al agregar un nuevo rating a la base de datos, se tendría que recalcular algunas de estas diferencias para este usuario. Esto significa que se puede tener otra base de datos con las diferencias entre items para los usuarios ya precalculados. Esto ayudaría en que no hay que hacer todos los cálculos todo de nuevo al momento de tener un nuevo rating, y solamente cambiar los ratings nuevos del usuario, y usar esta base de datos para generar una predicción.\\
Al tener un usuario que ha dado pocos ratings, el recomendador no le va a ir muy bien debido a que se necesita encontrar pares de items en el cual este usuario dió calificaciones y otros usuarios también dieron calificaciones. Podría pasar que un usuario ha dado solamente una calificación, el cual solo tiene ese rating. Lo que pasa en ese caso es que no se puede comparar con otros usarios (ya que nadie más ha visto ese item), y por ende, no se le puede predecir para otras películas.



# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [1]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    568      0 --:--:-- --:--:-- --:--:--   568
100 1546k  100 1546k    0     0  1567k      0 --:--:-- --:--:-- --:--:--  107M


In [2]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    783      0 --:--:-- --:--:-- --:--:--   783
100  385k  100  385k    0     0   552k      0 --:--:-- --:--:-- --:--:--  552k


In [3]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    930      0 --:--:-- --:--:-- --:--:--   930
100  230k  100  230k    0     0   370k      0 --:--:-- --:--:-- --:--:--  370k


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [4]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 234 kB 6.1 MB/s 


## Paso 3:

Hacemos los imports necesarios para este práctico.

In [5]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [6]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [7]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [9]:
# Y entrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:** \\
En el paper, lo que se trata de hacer es encontrar una función $f(x) = x + b$; en el cual $b$ es una constante, y $x$ es una variable que representa a valores de ratings. Para cada par de items, se trata de encontrar una función igual a la mencionada que busca predecir el valor de un rating de un item a base de los ratings del otro. Por lo tanto, al momento de ejecutar `train()`, se genera un modelo con todas estas funciones para poder predecir los ratings que haría un usuario a una película. Por lo tanto, al momento de querer predecir, solamente consultamos la función que está almacenada en este modelo, y nos da una predicción. 

## Paso 6:

Llego la hora de predecir el rating.

In [10]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.5898244380950928

In [11]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [17]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457].sort_values(by=['rating']).tail(15)

,userid,itemid,rating,timestamp
37311,457,176,5,882397542
37309,457,169,5,882396935
37308,457,162,5,882548793
37382,457,528,5,882397543
37326,457,200,5,882396799
37305,457,154,5,882397058
37304,457,151,5,882394010
37388,457,582,5,882548350
37389,457,588,5,882397411
37302,457,147,5,882395400


In [13]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?  

**Respuesta:** \\
Lo que puede estar pasando es que el modelo de SlopeOne no esté guardando el valor de un rating si es que existe, pero igual se calcula una función (anteriormente mencionado) que se usa para predecir su valor. Por lo tanto, al momento de querer predecir el valor de un rating ya existente, igual se va a recurrir al modelo el cual tiene una función precalculada gracias a los otros ratings de los datos, y este dará un valor (el cual puede estar cerca a ya existente, o no)\\
Podría pasar que a alguien le ha gustado las mismas películas que al resto, pero para una película en específico al resto le cargó pero a este usuario le encantó. Nuestro modelo tal vez predecería que no le gustaría, mientras que nosotros sabemos que ese no es el caso.

In [18]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.385906219482422
Prediction for ID:1168 : 4.206681251525879


## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [19]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [20]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:** \\
Al revisar algunos usuarios, se puede ver que hay unas películas en común. Por ejemplo, para los usuarios con id 457,69,128 y 150, se puede ver que hay varias películas en común. Esto puede estar pasando debido a que hay mucha gente viendo estas películas y dándole un rating mayor a las otras películas que ellos ven, y por lo tanto, al momento de predecir un rating de esta película a un usuario en específico, se tienda a predecir un rating mayor a los que ese usuario tiene (y finalmente, tener una mayor probabilidad a recomendar esa película). 

In [42]:
# Escribe el nuevo codigo aqui
for u in ["65", "69", "130"]:
  reclist_slopeone = mySlopeOne.recommend(u, 7)
  recmovies_slopeone = np.array(reclist_slopeone).astype(int)

  print("         Peliculas de usuario", u)
  print(info_file.loc[recmovies_slopeone]['title'])
  print("")

         Peliculas de usuario 65
movieid
1589              Schizopolis (1996)
1656              Little City (1998)
1651    Spanish Prisoner, The (1997)
1650         Butcher Boy, The (1998)
1645         Butcher Boy, The (1998)
1642        Some Mother's Son (1996)
1636      Brothers in Trouble (1995)
Name: title, dtype: object

         Peliculas de usuario 69
movieid
1653    Entertaining Angels: The Dorothy Day Story (1996)
1064                                     Crossfire (1947)
1651                         Spanish Prisoner, The (1997)
1650                              Butcher Boy, The (1998)
1645                              Butcher Boy, The (1998)
1642                             Some Mother's Son (1996)
1636                           Brothers in Trouble (1995)
Name: title, dtype: object

         Peliculas de usuario 130
movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   

## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:** \\
Lo que hice fue crear dos listas de recomendaciones utilizando estos dos algoritmos para los top 100 items, y después encontré a los items que tenían en común (además de las posiciones en estas listas de recomendaciones). Al parecer no eran tantos; solo tenían 5 películas en común. \\
Most Popular solo le importa el total de ratings que tenga una película, sin importar si el rating final de esa película sea buena o no; mientras que en slope one se busca la diferencia entre pares de ratings, y por lo tanto, las predicciones se basan en encontrar películas que comparadas con otras nos de el mejor predicción. \\
Supuestamente, slope one debería dar recomendaciones más personalizadas, ya que hay que considerar los ratings que ha hecho un usuario al compararlo con otros usuarios. Para probar esto, hice la misma comparación de modelos de Most Popular y Slope one, pero con otro usuario (usuario ID: 45). Posteriormente a esto, encuentro los id de las películas en común en las recomendaciones hechas por Most Popular para los dos usuarios, y lo mismo para Slope One. Se puede ver que Slope One recomendó 76 películas en común entre los usuarios, y Most Popular 81 películas en común, lo que valida mi hipótesis. 

In [50]:
# Primero creamos el modelo para Most Popular
most_popular = pyreclab.MostPopular(dataset='u1.base',
                   dlmchar=b'\t',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)

most_popular.train() # y lo "entrenamos"

In [75]:
user_id = "44"
top_n = 100

print('Lista de items según ID para most popular:', most_popular.recommend(user_id, top_n, includeRated=False))
print('Lista de items según ID para slope one:', mySlopeOne.recommend(user_id, top_n))

same = []

MPList1 =  most_popular.recommend(user_id, top_n, includeRated=False)
SOList1 = mySlopeOne.recommend(user_id, top_n)

for i in MPList1:
  if i in SOList1:
    same.append([i, MPList1.index(i), SOList1.index(i)])
  
print("Ids de películas en común entre los modelos")
print(same)

Lista de items según ID para most popular: ['50', '100', '181', '286', '1', '121', '300', '127', '7', '98', '172', '56', '237', '117', '222', '204', '313', '173', '79', '151', '210', '269', '69', '748', '96', '22', '9', '168', '195', '328', '302', '118', '183', '276', '202', '423', '25', '15', '234', '64', '28', '176', '275', '289', '268', '82', '135', '89', '111', '238', '357', '196', '12', '153', '125', '144', '228', '333', '475', '245', '323', '496', '194', '197', '483', '185', '282', '182', '322', '180', '71', '427', '143', '8', '161', '179', '11', '215', '187', '235', '4', '95', '678', '200', '88', '603', '508', '597', '208', '134', '271', '307', '272', '588', '393', '211', '230', '403', '474', '250']
Lista de items según ID para slope one: ['1656', '1064', '1625', '1599', '1512', '1536', '1500', '1463', '1367', '1293', '814', '48', '47', '46', '45', '44', '43', '41', '39', '36', '35', '33', '29', '28', '27', '16', '14', '13', '12', '10', '1642', '1651', '1650', '1645', '1636', '1

In [76]:
user_id = "45"
top_n = 100

print('Lista de items según ID para most popular:', most_popular.recommend(user_id, top_n, includeRated=False))
print('Lista de items según ID para slope one:', mySlopeOne.recommend(user_id, top_n))

same = []

MPList2 =  most_popular.recommend(user_id, top_n, includeRated=False)
SOList2 = mySlopeOne.recommend(user_id, top_n)

for i in MPList2:
  if i in SOList2:
    same.append([i, MPList2.index(i), SOList2.index(i)])
  
print("Ids de películas en común entre los modelos")
print(same)

Lista de items según ID para most popular: ['50', '100', '258', '181', '286', '288', '1', '121', '300', '127', '7', '98', '172', '56', '237', '204', '313', '173', '405', '79', '151', '210', '269', '69', '748', '96', '22', '9', '168', '195', '328', '302', '257', '118', '183', '276', '202', '318', '216', '423', '25', '15', '234', '64', '28', '742', '176', '275', '191', '289', '268', '82', '135', '89', '111', '238', '357', '196', '12', '153', '125', '186', '144', '228', '333', '475', '97', '546', '245', '70', '323', '496', '471', '301', '132', '194', '483', '568', '185', '282', '655', '182', '180', '427', '143', '8', '161', '179', '11', '385', '187', '435', '235', '4', '95', '200', '88', '603', '508', '597']
Lista de items según ID para slope one: ['1064', '1651', '1650', '1645', '1636', '1585', '1623', '1472', '1599', '1512', '1450', '1536', '1523', '1467', '1463', '1449', '1418', '1201', '814', '48', '47', '46', '45', '44', '42', '41', '39', '36', '35', '33', '31', '29', '28', '27', '16

In [81]:
sameMP = []
for i in MPList2:
  if i in MPList1:
    sameMP.append(i)

sameSO = []
for i in SOList2:
  if i in SOList1:
    sameSO.append(i)

print(f"ID de {len(sameMP)} peliculas iguales en Most popular para usarios 44 y 45")
print(sameMP)

print(f"ID de {len(sameSO)} peliculas iguales en Slope One para usarios 44 y 45")
print(sameSO)

ID de 81 peliculas iguales en Most popular para usarios 44 y 45
['50', '100', '181', '286', '1', '121', '300', '127', '7', '98', '172', '56', '237', '204', '313', '173', '79', '151', '210', '269', '69', '748', '96', '22', '9', '168', '195', '328', '302', '118', '183', '276', '202', '423', '25', '15', '234', '64', '28', '176', '275', '289', '268', '82', '135', '89', '111', '238', '357', '196', '12', '153', '125', '144', '228', '333', '475', '245', '323', '496', '194', '483', '185', '282', '182', '180', '427', '143', '8', '161', '179', '11', '187', '235', '4', '95', '200', '88', '603', '508', '597']
ID de 75 peliculas iguales en Slope One para usarios 44 y 45
['1064', '1651', '1650', '1645', '1636', '1585', '1623', '1472', '1599', '1512', '1450', '1536', '1523', '1467', '1463', '1449', '1418', '1201', '814', '48', '47', '46', '45', '44', '41', '39', '36', '35', '33', '29', '28', '27', '16', '14', '13', '12', '10', '1500', '1642', '1398', '1639', '1656', '1589', '1080', '1592', '1389', '1